In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

from keras.models import Model
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pickle
import keras
import tensorflow as tf
from keras import backend as K
import sys
import time
sys.path.append(os.path.abspath('../'))
import helpers
from helpers.data_generator import process_data, AutoEncoderDataGenerator, DataGenerator
from helpers.custom_losses import normed_mse, mean_diff_sum_2, max_diff_sum_2, mean_diff2_sum2, max_diff2_sum2, denorm_loss, hinge_mse_loss, percent_correct_sign, baseline_MAE
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from time import strftime, localtime
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import copy
from tqdm import tqdm_notebook
from helpers.normalization import normalize, denormalize, renormalize
import scipy
from keras.utils.vis_utils import model_to_dot
from IPython.display import Image, display
from helpers.custom_init import downsample
from helpers.custom_reg import groupLasso
import helpers
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
os.chdir(os.path.expanduser('/home/aiqtidar/run_results_04_10/'))
files = [foo for foo in os.listdir() if '0_params.pkl' in foo]
print("Matching files at {} are : {}".format(os.getcwd(),files[0]))

Matching files at /home/aiqtidar/run_results_04_10 are : final_model-autoencoder_SET-dense_SDT-dense_CET-dense_CDT-dense_profiles-dens-temp-q_EFIT01-rotation-press_EFIT01_act-target_density-pinj-tinj-curr_target_LB-0_LA-6_10Apr21-17-25_Scenario-0_params.pkl


In [6]:
scenarios_dict = {}

base_path = '/home/aiqtidar/run_results_04_10/'

# model_name = 'final_model-autoencoder_SET-dense_SDT-dense_CET-dense_CDT-dense_profiles-temp-dens-ffprime_EFIT02-press_EFIT02-q_EFIT02_act-pinj-curr-tinj-gasA_LB-0_LA-6_09Apr21-01-53_Scenario-'
# model_extension = '.h5'

scenario_name = 'final_model-autoencoder_SET-dense_SDT-dense_CET-dense_CDT-dense_profiles-temp-dens-ffprime_EFIT02-press_EFIT02-q_EFIT02_act-pinj-curr-tinj-gasA_LB-0_LA-6_09Apr21-02-50_Scenario-'
scenario_extension = '_params.pkl'
# scenario_path = base_path + scenario_name + str(number) + scenario_extension

for number in range(0,8):
    files = [foo for foo in os.listdir() if (str(number) + scenario_extension) in foo]
    scenario_path = base_path + files[0]
    with open(scenario_path, 'rb') as f:
        scenario = pickle.load(f, encoding='latin1')
    scenarios_dict[number] = scenario



In [7]:
scenarios_dict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7])

In [10]:
scenarios_dict[0]['state_encoder_kwargs']['layer_scale']

1

In [141]:
scenarios_dict[0]['history'].keys()

dict_keys(['val_loss', 'val_u_residual_loss', 'val_x_residual_loss', 'val_linear_system_residual_loss', 'val_u_residual_mean_squared_error', 'val_u_residual_mean_absolute_error', 'val_x_residual_mean_squared_error', 'val_x_residual_mean_absolute_error', 'val_linear_system_residual_mean_squared_error', 'val_linear_system_residual_mean_absolute_error', 'loss', 'u_residual_loss', 'x_residual_loss', 'linear_system_residual_loss', 'u_residual_mean_squared_error', 'u_residual_mean_absolute_error', 'x_residual_mean_squared_error', 'x_residual_mean_absolute_error', 'linear_system_residual_mean_squared_error', 'linear_system_residual_mean_absolute_error', 'lr', 'end_times', 'epoch_times'])

In [137]:
scenarios_dict[0]['history']['loss'][-1]

0.00025391113712171394

# Analysis

## Run Results 04_08

### Summary :
- Model 0 for x_weight, u_weight = 1,1
- Models 16 and 20 for x_weight, u_weight = 1, 0.1

## Approach

Group models together based on u-weight and x-weight. There are two possibilities:

[{'x_weight':1, 'u_weight':1}, {'x_weight':1, 'u_weight':0.1}]

Errors are directly comparable within the two, since the loss functions are identical.

Metrics that are evaluated are

- 1: abs(train/val) -> The smaller this number the better
- 2: min(loss) -> Minimum training loss
- 3: min(val_loss) -> Minimum val loss
- 4: loss[-1] -> Last training loss
- 5: val_loss[-1] -> Last val loss



In [171]:
u_weight_l = {}
u_weight_h = {}

indices_l = []
min_val_loss_l = []
min_train_loss_l = []
fin_val_loss_l = []
fin_train_loss_l = []


indices_h = []
min_val_loss_h = []
min_train_loss_h = []
fin_val_loss_h = []
fin_train_loss_h = []


for i in range(0,len(scenarios_dict)):
    if scenarios_dict[i]['u_weight'] < 1:
        indices_l.append(i)
        min_train_loss_l.append(np.min(scenarios_dict[i]['history']['loss']))
        min_val_loss_l.append(np.min(scenarios_dict[i]['history']['val_loss']))
        fin_val_loss_l.append(scenarios_dict[i]['history']['val_loss'][-1])
        fin_train_loss_l.append(scenarios_dict[i]['history']['loss'][-1])
    else:
        indices_h.append(i)
        min_train_loss_h.append(np.min(scenarios_dict[i]['history']['loss']))
        min_val_loss_h.append(np.min(scenarios_dict[i]['history']['val_loss']))
        fin_val_loss_h.append(scenarios_dict[i]['history']['val_loss'][-1])
        fin_train_loss_h.append(scenarios_dict[i]['history']['loss'][-1])


min_val_loss_l = np.array(min_val_loss_l)
min_train_loss_l = np.array(min_train_loss_l)
fin_val_loss_l = np.array(fin_val_loss_l)
fin_train_loss_l = np.array(fin_train_loss_l)

min_val_loss_h = np.array(min_val_loss_h)
min_train_loss_h = np.array(min_train_loss_h)
fin_val_loss_h = np.array(fin_val_loss_h)
fin_train_loss_h = np.array(fin_train_loss_h)
        
print("Results for 'h': \n")
print("min_val_loss_h = {} \n min_train_loss_h = {} \n fin_val_loss_h = {} \n fin_train_loss_h = {} \n min_train/val = {} \n".format(np.argmin(min_train_loss_h),np.argmin(min_val_loss_h),np.argmin(fin_val_loss_h),np.argmin(fin_train_loss_h), (np.abs(1 - fin_train_loss_h/fin_val_loss_h))))
print("Best model is thus: {} \n".format(indices_h[0]))

print("Results for 'l': \n")
print("min_val_loss_l = {} \n min_train_loss_l = {} \n fin_val_loss_l = {} \n fin_train_loss_l = {} \n min_train/val = {} \n".format(np.argmin(min_train_loss_l),np.argmin(min_val_loss_l),np.argmin(fin_val_loss_l),np.argmin(fin_train_loss_l), (np.abs(1 - fin_train_loss_h/fin_val_loss_l))))
print("Best models for l are thus: {} and {}".format(indices_l[0], indices_l[4]))

Results for 'h': 

min_val_loss_h = 0 
 min_train_loss_h = 0 
 fin_val_loss_h = 0 
 fin_train_loss_h = 0 
 min_train/val = [0.00277504 0.00098954 0.01542861 0.01880641 0.00082989 0.00095373
 0.04452105 0.0137315  0.00087665 0.00097745 0.02293515 0.00313365
 0.00090233 0.00098387 0.02083868 0.00244364] 

Best model is thus: 0 

Results for 'l': 

min_val_loss_l = 12 
 min_train_loss_l = 12 
 fin_val_loss_l = 12 
 fin_train_loss_l = 12 
 min_train/val = [  0.15495161   8.41870202   0.20352213   0.73625215 135.57701437
 139.08616135   2.28790087   5.41955359 101.17040954   8.54937671
   0.75244605   0.17104445 148.29704355   8.45137169   0.53474846
   2.6811059 ] 

Best models for l are thus: 16 and 20


for h, Model 0 seems to be optimal

for l, Models 16 and 20 are interesting

In [178]:
min_val_loss_h

array([0.00025321, 0.02429986, 0.00513845, 0.00348803, 0.02422663,
       0.02443801, 0.00521306, 0.00494815, 0.02437353, 0.02430904,
       0.00892593, 0.00968264, 0.02436592, 0.02423983, 0.00625088,
       0.01731732])

In [175]:
scenarios_dict[20];

## Run Results 04_10

### Summary:

- For encoder layer scale = 2, the best model is 7
- For encoder layer scale = 1, the best model is 2

### Approach

Sort by layer scale

Metrics that are evaluated are

- 1: abs(train/val) -> The smaller this number the better
- 2: min(loss) -> Minimum training loss
- 3: min(val_loss) -> Minimum val loss
- 4: loss[-1] -> Last training loss
- 5: val_loss[-1] -> Last val loss


In [13]:
u_weight_l = {}
u_weight_h = {}

indices_l = []
min_val_loss_l = []
min_train_loss_l = []
fin_val_loss_l = []
fin_train_loss_l = []


indices_h = []
min_val_loss_h = []
min_train_loss_h = []
fin_val_loss_h = []
fin_train_loss_h = []


for i in range(0,len(scenarios_dict)):
    if scenarios_dict[i]['state_encoder_kwargs']['layer_scale'] < 2:
        indices_l.append(i)
        min_train_loss_l.append(np.min(scenarios_dict[i]['history']['loss']))
        min_val_loss_l.append(np.min(scenarios_dict[i]['history']['val_loss']))
        fin_val_loss_l.append(scenarios_dict[i]['history']['val_loss'][-1])
        fin_train_loss_l.append(scenarios_dict[i]['history']['loss'][-1])
    else:
        indices_h.append(i)
        min_train_loss_h.append(np.min(scenarios_dict[i]['history']['loss']))
        min_val_loss_h.append(np.min(scenarios_dict[i]['history']['val_loss']))
        fin_val_loss_h.append(scenarios_dict[i]['history']['val_loss'][-1])
        fin_train_loss_h.append(scenarios_dict[i]['history']['loss'][-1])


min_val_loss_l = np.array(min_val_loss_l)
min_train_loss_l = np.array(min_train_loss_l)
fin_val_loss_l = np.array(fin_val_loss_l)
fin_train_loss_l = np.array(fin_train_loss_l)

min_val_loss_h = np.array(min_val_loss_h)
min_train_loss_h = np.array(min_train_loss_h)
fin_val_loss_h = np.array(fin_val_loss_h)
fin_train_loss_h = np.array(fin_train_loss_h)
        
print("Results for 'h': \n")
print("min_val_loss_h = {} \n min_train_loss_h = {} \n fin_val_loss_h = {} \n fin_train_loss_h = {} \n min_train/val = {} \n".format(np.argmin(min_train_loss_h),np.argmin(min_val_loss_h),np.argmin(fin_val_loss_h),np.argmin(fin_train_loss_h), (np.abs(1 - fin_train_loss_h/fin_val_loss_h))))
print("Best model is thus: {} \n".format(indices_h[3]))

print("Results for 'l': \n")
print("min_val_loss_l = {} \n min_train_loss_l = {} \n fin_val_loss_l = {} \n fin_train_loss_l = {} \n min_train/val = {} \n".format(np.argmin(min_train_loss_l),np.argmin(min_val_loss_l),np.argmin(fin_val_loss_l),np.argmin(fin_train_loss_l), (np.abs(1 - fin_train_loss_h/fin_val_loss_l))))
print("Best models for l are thus: {}".format(indices_l[1]))

Results for 'h': 

min_val_loss_h = 3 
 min_train_loss_h = 3 
 fin_val_loss_h = 3 
 fin_train_loss_h = 3 
 min_train/val = [0.00566873 0.00410943 0.00908795 0.00797236] 

Best model is thus: 7 

Results for 'l': 

min_val_loss_l = 1 
 min_train_loss_l = 1 
 fin_val_loss_l = 1 
 fin_train_loss_l = 1 
 min_train/val = [0.47358025 0.52318257 0.063628   0.13391871] 

Best models for l are thus: 2
